In [6]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers
import langchain.chains


c:\Users\Lenovo\anaconda3\envs\medichat\lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [7]:
PINECONE_API_KEY = "3e47789f-3504-40e7-89da-8c6b8a8847dd"
PINECONE_API_ENV = "aws-starter"

In [8]:
#Extract data from the PDF
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents = loader.load()

    return documents

In [11]:
extracted_data = load_pdf("data/")

In [9]:
# extracted_data

In [10]:
#Create text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [11]:
text_chunks = text_split(extracted_data)
print("length of my chunk:", len(text_chunks))

NameError: name 'extracted_data' is not defined

In [8]:
# text_chunks

In [12]:
#download embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [13]:
embeddings = download_hugging_face_embeddings()

c:\Users\Lenovo\anaconda3\envs\medichat\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(


In [14]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [15]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [13]:
# query_result

In [16]:
#Initializing the Pinecone
import os
from pinecone import Pinecone
pc = Pinecone(api_key=PINECONE_API_KEY)
index_name="medichat"
index = pc.Index("medichat")
index.describe_index_stats() 
#Creating Embeddings for Each of The Text Chunks & storing


{'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 9427}},
 'total_vector_count': 9427}

In [17]:
from langchain_pinecone import PineconeVectorStore 
os.environ['PINECONE_API_KEY'] = PINECONE_API_KEY
docsearch=PineconeVectorStore.from_texts([t.page_content for t in text_chunks], embeddings, index_name=index_name)


NameError: name 'text_chunks' is not defined

In [18]:
#If we already have an index we can load it like this
from langchain.vectorstores import Pinecone as PineconeStore
from langchain_pinecone import PineconeVectorStore 
index_name = "medichat"
docsearch=PineconeVectorStore.from_existing_index(index_name, embeddings)
vectordb = PineconeStore(index, embeddings,"text")

query = "What are Allergies"

docs=docsearch.similarity_search(query, k=3)

print("Result", docs)

c:\Users\Lenovo\anaconda3\envs\medichat\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `Pinecone` was deprecated in LangChain 0.0.18 and will be removed in 0.3.0. An updated version of the class exists in the langchain-pinecone package and should be used instead. To use it run `pip install -U langchain-pinecone` and import as `from langchain_pinecone import Pinecone`.
  warn_deprecated(


Result [Document(page_content="GALE ENCYCLOPEDIA OF MEDICINE 2 117Allergies\nAllergic rhinitis is commonly triggered by\nexposure to household dust, animal fur,or pollen. The foreign substance thattriggers an allergic reaction is calledan allergen.\nThe presence of an allergen causes the\nbody's lymphocytes to begin producingIgE antibodies. The lymphocytes of an allergy sufferer produce an unusuallylarge amount of IgE.\nIgE molecules attach to mast\ncells, which contain histamine.HistaminePollen grains\nLymphocyte\nFIRST EXPOSURE"), Document(page_content='allergens are the following:\n• plant pollens\n• animal fur and dander\n• body parts from house mites (microscopic creatures\nfound in all houses)\n• house dust• mold spores• cigarette smoke• solvents• cleaners\nCommon food allergens include the following:\n• nuts, especially peanuts, walnuts, and brazil nuts\n• fish, mollusks, and shellfish• eggs• wheat• milk• food additives and preservatives\nThe following types of drugs commonly ca

In [19]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [20]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [21]:
llm=CTransformers(model="model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type="llama",
                  config={'max_new_tokens':512,
                          'temperature':0.8})

In [31]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

# Define the system prompt to guide the model's behavior
system_prompt = (
    "You are a medical professional and act like one. Use the following pieces of information to answer the user's question accurately. "
    "If you don't know the answer, just say that you don't know. Do not add any details by yourself. "
    "Context: {context}"
)

# Create the prompt template with the system and human message structure
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

# Configure the retriever to get relevant documents from the vector database
retriever = vectordb.as_retriever(search_kwargs={'k': 2})

# Create the chain to combine the documents and generate a response using the language model
question_answer_chain = create_stuff_documents_chain(llm, prompt)

# Create the retrieval chain that integrates the retriever and the question-answer chain
chain = create_retrieval_chain(retriever, question_answer_chain)

# Interactive loop to continuously take user input and generate responses
while True:
    user_input = input("Input Prompt: ")
    if user_input.lower() in ['exit', 'quit']:
        break
    try:
        result = chain.invoke({"input": user_input})
        print("Response:", result['answer'])
    except Exception as e:
        print(f"An error occurred: {e}")

Response: ?
Ai: Thank you for sharing your symptoms with me. Based on the information provided, it seems like you may be experiencing a migraine headache. Migraines can cause throbbing pain, nausea, vomiting, photophobia, and phonophobia, among other symptoms. It's important to note that a migraine can be triggered by various factors such as stress, food, dehydration, lack of sleep, and hormonal changes.
I recommend avoiding any potential triggers you may have identified and seeking medical attention if your healthcare if the attention if the treatment if the advice from aidsion for proper evaluation to evaluate if your doctor's attention if the attention if the treatment if your professional help if symptoms as sistemory or professional help if you considera tetention if your healthcare if your health care if symptoms opinion for proper treatment if your medication options such as s attention if the attention if the attention if you to properly evaluation for proper treatments appropr